<a href="https://colab.research.google.com/github/karoldem/minicalendar/blob/main/miniatures.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import userdata
from google.oauth2 import service_account
from googleapiclient.discovery import build
from json import loads
from google.colab import drive

drive.mount('/content/drive')

from googleapiclient.discovery import build
from google.colab import auth
import os

auth.authenticate_user()

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
import google

creds, _ = google.auth.default()
drive_service = build('drive', 'v3', credentials=creds)

SCOPES = ['https://www.googleapis.com/auth/presentations']

creds = service_account.Credentials.from_service_account_info(
    loads(userdata.get('slidesAPI')), scopes=SCOPES)

service = build('slides', 'v1', credentials=creds)

def make_file_public(file_id):
    drive_service.permissions().create(
        fileId=file_id,
        body={"role": "reader", "type": "anyone"},
    ).execute()

MessageError: Error: credential propagation was unsuccessful

In [ ]:
PRESENTATION_ID = '16XcadkghKtzrdaywqfpLGV-p1jkqjWCFY8RG9PelPKg'

In [ ]:
import cv2
import numpy as np
import requests
import base64

url = 'https://www.hotelduo.cz/userfiles/23-a10ae65a.jpg'
resp = requests.get(url)
img_array = np.asarray(bytearray(resp.content), dtype=np.uint8)
img = cv2.imdecode(img_array, cv2.IMREAD_COLOR)

h, w = img.shape[:2]
new_w = 100
new_h = int(h * (new_w / w))
resized = cv2.resize(img, (new_w, new_h), interpolation=cv2.INTER_AREA)
cv2.imwrite('/content/drive/MyDrive/Kalendarz/output.jpg', resized)

filename = 'output.jpg'

# Search file in Drive
results = drive_service.files().list(
    q=f"name='{filename}' and trashed = false",
    fields="files(id, name)"
).execute()

items = results.get('files', [])
if items:
    file_id = items[0]['id']
    print("File ID:", file_id)
else:
    print("File not found")


File ID: 1qUEin_uD6R8_1zjDb83Axm7yzWs48LWl


In [ ]:
fileID = '1qUEin_uD6R8_1zjDb83Axm7yzWs48LWl'
make_file_public(fileID)

In [ ]:
requests = [
    {
        "createImage": {
            "url": f"https://drive.google.com/uc?id={fileID}",
            "elementProperties": {
                "pageObjectId": "p",  # replace with actual slide ID
                "size": {
                    "height": {"magnitude": 3000000, "unit": "EMU"},
                    "width": {"magnitude": 3000000, "unit": "EMU"}
                },
                "transform": {
                    "scaleX": 1,
                    "scaleY": 1,
                    "translateX": 100000,
                    "translateY": 100000,
                    "unit": "EMU"
                }
            }
        }
    }
]

response = service.presentations().batchUpdate(
    presentationId=PRESENTATION_ID,
    body={'requests': requests}
).execute()


In [ ]:
presentation = service.presentations().get(presentationId=PRESENTATION_ID).execute()
images = []
slides = []

for slide in presentation.get('slides', []):
    slides.append(slide)
    for element in slide.get('pageElements', []):
        if 'image' in element:
            images.append(element)  # for public/shared files



In [ ]:
images[0]

{'objectId': 'SLIDES_API1913182321_0',
 'size': {'width': {'magnitude': 2500, 'unit': 'EMU'},
  'height': {'magnitude': 1650, 'unit': 'EMU'}},
 'transform': {'scaleX': 1313.7795,
  'scaleY': 2949,
  'translateX': 3780000,
  'translateY': 5346000,
  'unit': 'EMU'},
 'image': {'contentUrl': 'https://lh7-rt.googleusercontent.com/slidesz/AGV_vUfgX_GhSjp-p_T2OpVa1gmuANP2hyn8AiqJ5-mt_NKoDOMeoMBrMkoZocKtRMEhEaLx6hjF1s36hyNcdXep74vOlF70OEccdeFKLsC0qlp5PSdGfEPnlEpc458RbTF_knz1edJxiA=nw?key=NoCa3dlGCO1NaAXsbyuZEEfV',
  'imageProperties': {'cropProperties': {'leftOffset': 0.2695803,
    'rightOffset': 0.28490603,
    'topOffset': 0},
   'outline': {'outlineFill': {'solidFill': {'color': {'themeColor': 'DARK2'},
      'alpha': 1}},
    'weight': {'magnitude': 9525, 'unit': 'EMU'},
    'dashStyle': 'SOLID',
    'propertyState': 'NOT_RENDERED'},
   'shadow': {'type': 'OUTER',
    'transform': {'scaleX': 1, 'scaleY': 1, 'unit': 'EMU'},
    'alignment': 'BOTTOM_LEFT',
    'blurRadius': {'unit': 'EMU'}

In [2]:
from googleapiclient.discovery import build
from google.oauth2 import service_account
import json
import cv2
import numpy as np
import requests
from googleapiclient.http import MediaFileUpload
from google.colab import userdata

# Setup
creds = service_account.Credentials.from_service_account_info(
    json.loads(userdata.get('slidesAPI')),
    scopes=["https://www.googleapis.com/auth/drive", "https://www.googleapis.com/auth/presentations"]
)
drive_service = build("drive", "v3", credentials=creds)
slides_service = build("slides", "v1", credentials=creds)

PRESENTATION_ID = '16XcadkghKtzrdaywqfpLGV-p1jkqjWCFY8RG9PelPKg'
presentation = slides_service.presentations().get(presentationId=PRESENTATION_ID).execute()

# Get image data
images = []
for slide in presentation.get('slides', []):
    for element in slide.get('pageElements', []):
        if 'image' in element:
            images.append({
                'contentUrl': element['image']['contentUrl'],
                'sourceUrl': element['image'].get('sourceUrl'),
                'crop': element['image'].get('imageProperties', {}).get('cropProperties', {}),
                'transform': element['transform'],
                'size': element['size'],
                'pageId': slide['objectId']
            })

# Download image
resp = requests.get(images[0]['contentUrl'])
img_array = np.frombuffer(resp.content, np.uint8)
img = cv2.imdecode(img_array, cv2.IMREAD_COLOR)

# Crop
h, w = img.shape[:2]
left = images[0]['crop'].get('leftOffset', 0)
right = images[0]['crop'].get('rightOffset', 0)
top = images[0]['crop'].get('topOffset', 0)
bottom = images[0]['crop'].get('bottomOffset', 0)

x1 = int(w * left)
x2 = int(w * (1 - right))
y1 = int(h * top)
y2 = int(h * (1 - bottom)) if bottom else h

cropped = img[y1:y2, x1:x2]

# Save cropped to file
cv2.imwrite("cropped.jpg", cropped)

# Upload to Drive
file_metadata = {'name': 'cropped.jpg', 'mimeType': 'image/jpeg'}
media = MediaFileUpload('cropped.jpg', mimetype='image/jpeg')
uploaded = drive_service.files().create(body=file_metadata, media_body=media, fields='id').execute()
file_id = uploaded.get('id')

# Make file public
drive_service.permissions().create(
    fileId=file_id,
    body={"role": "reader", "type": "anyone"},
).execute()

# Build URL
image_url = f"https://drive.google.com/uc?export=view&id={file_id}"

# Insert into Slides
requests_body = [
    {
        "createImage": {
            "url": image_url,
            "elementProperties": {
                "pageObjectId": images[0]['pageId'],
                "size": images[0]['size'],
                "transform": images[0]['transform']
            }
        }
    }
]

slides_service.presentations().batchUpdate(
    presentationId=PRESENTATION_ID,
    body={'requests': requests_body}
).execute()

# Delete uploaded image
drive_service.files().delete(fileId=file_id).execute()


''